In [1]:
import pandas as pd
import numpy as np

In [2]:
target_classes = [
    'Above ear - pull hair',
    'Forehead - pull hairline',
    'Forehead - scratch',
    'Eyebrow - pull hair',
    'Eyelash - pull hair',
    'Neck - pinch skin',
    'Neck - scratch',
    'Cheek - pinch skin'
]
class_map = {cls: i for i, cls in enumerate(target_classes)}
class_map

{'Above ear - pull hair': 0,
 'Forehead - pull hairline': 1,
 'Forehead - scratch': 2,
 'Eyebrow - pull hair': 3,
 'Eyelash - pull hair': 4,
 'Neck - pinch skin': 5,
 'Neck - scratch': 6,
 'Cheek - pinch skin': 7}

In [10]:
from tensorflow.keras.models import load_model
imuex_model_best = load_model("imuex_model_9_1.h5")


2025-09-01 20:50:02.941050: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-01 20:50:02.941298: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-01 20:50:02.972002: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-01 20:50:03.781585: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [7]:
df_test = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/test.csv')

In [4]:
imu_features =  [
 'acc_x',
 'acc_y',
 'acc_z',
 'rot_w',
 'rot_x',
 'rot_y',
 'rot_z',
 'acc_magnitude',
 'jerk_x',
 'jerk_y',
 'jerk_z',
 'jerk_magnitude']

In [5]:
import numpy as np
import pandas as pd

def prepare_submission_sequences(df,columns,max_steps=400):

    nan_columns_X = df.columns[df.isnull().any()].tolist()
    print("Columns in X with NaNs Before:\n", nan_columns_X)
    df = df.infer_objects(copy=False)
    df[df.select_dtypes(include=["number"]).columns] = df.select_dtypes(include=["number"]).interpolate(method="linear", axis=0)
    nan_columns_X = df.columns[df.isnull().any()].tolist()
    print("Columns in X with NaNs After:\n", nan_columns_X)
    
    df = df.sort_values(by=["sequence_id", "sequence_counter"], 
                    ascending=[True, True]).reset_index(drop=True)

  
    df["acc_magnitude"] = np.sqrt(df["acc_x"]**2 + df["acc_y"]**2 + df["acc_z"]**2)

    df["jerk_x"] = df["acc_x"].diff()   # difference between consecutive values
    df["jerk_y"] = df["acc_y"].diff()
    df["jerk_z"] = df["acc_z"].diff()

    # magnitude of jerk
    df["jerk_magnitude"] = np.sqrt(df["jerk_x"]**2 + df["jerk_y"]**2 + df["jerk_z"]**2)

    # fill NaN from diff() with 0
    df.fillna(0, inplace=True)

    # Drop unnecessary columns early
    df = df.drop(columns=["sequence_counter"]) 
    
    n_features = len(columns)

    
    # Find number of unique sequences
    sequence_ids = df["sequence_id"].unique()
    n_sequences = len(sequence_ids)

    # Preallocate final arrays
    X = np.zeros((n_sequences, max_steps, n_features), dtype=np.float32)
    Y = np.zeros((n_sequences,), dtype=np.float32)  
    for i, seq_id in enumerate(sequence_ids):
        if i % 100 ==0:
            print(f"Processing {i}/{n_sequences}")

        #imu
        seq = df[df["sequence_id"] == seq_id][columns].values.astype(np.float32)
        seq_len = len(seq)
        if seq_len > max_steps:
            seq = seq[-max_steps:]
        X[i, :seq_len, :] = seq
    
    return X

In [9]:
x_test = prepare_submission_sequences(df_test,imu_features)
x_test.shape

Columns in X with NaNs Before:
 []
Columns in X with NaNs After:
 []
Processing 0/2


(2, 400, 12)

In [11]:
prob = imuex_model_best.predict(x_test)
prob

/home/ankur/Desktop/ML_DL_Projects/ml_env/lib/python3.12/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step


array([[8.81773699e-03, 7.80835524e-02, 1.23726904e-01, 1.89436421e-01,
        2.78424680e-01, 2.33271699e-02, 1.76304411e-02, 1.19127281e-01,
        1.61425859e-01],
       [1.44980818e-01, 3.71901714e-03, 3.59731086e-04, 6.56306297e-02,
        1.77304521e-01, 3.48164767e-01, 1.13751493e-01, 1.46078810e-01,
        1.01682872e-05]], dtype=float32)

In [14]:
classpred = np.argmax(prob,axis=1)

In [16]:
if (classpred[0]==8):
    print('Text on phone')
print(target_classes[classpred[0]])

Eyelash - pull hair
